In [1]:
import grpc
import src.partner_api2_pb2_grpc as api
from src.partner_api2_pb2 import *
from src.emporia_conection import data_extract
from src.db_verify import checking_data
from src.transforming import transforming
from dateutil.relativedelta import relativedelta 
from datetime import datetime  
import pandas as pd
import time, glob